# 02 · Modeling, Calibration, and Decision Curves

In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np
import joblib

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.utils.class_weight import compute_class_weight

from src.modeling import build_preprocessor, build_models
from src.metrics import compute_metrics, reliability_curve
from src.dca import decision_curve
from src.plotting import plot_reliability, plot_dca

PROC = Path('data/processed')
train_df = pd.read_parquet(PROC / 'split_train.parquet')
val_df   = pd.read_parquet(PROC / 'split_valid.parquet')
test_df  = pd.read_parquet(PROC / 'split_test.parquet')

# === 1) Columns ===
target = 'severity_bin'
y_train = train_df[target]
y_val   = val_df[target]
y_test  = test_df[target]

feature_candidates = [c for c in train_df.columns if c not in {target, 'patient_id'}]
# Example numeric/categorical split (adjust per your columns)
num_cols = [c for c in feature_candidates if train_df[c].dtype != 'object']
cat_cols = [c for c in feature_candidates if train_df[c].dtype == 'object']

pre = build_preprocessor(num_cols, cat_cols)
models = build_models()

results = []
fitted = {}

for name, base_model in models.items():
    from sklearn.pipeline import Pipeline
    pipe = Pipeline([('pre', pre), ('model', base_model)])

    # Optional: handle imbalance via class_weight (for logreg)
    if hasattr(base_model, 'class_weight') and len(np.unique(y_train))>2:
        # for multi-class, sklearn expects 'balanced' or dict per class
        base_model.set_params(class_weight='balanced')

    pipe.fit(train_df[feature_candidates], y_train)
    fitted[name] = pipe

    # Validation predictions
    if hasattr(base_model, 'predict_proba'):
        y_val_prob = pipe.predict_proba(val_df[feature_candidates])
        # For multi-class metrics, compute macro AUROC
        auroc = roc_auc_score(y_val, y_val_prob, multi_class='ovr')
    else:
        # fallback using decision function with a softmax-like transform
        from sklearn.preprocessing import MinMaxScaler
        scores = pipe.decision_function(val_df[feature_candidates])
        scaler = MinMaxScaler()
        y_val_prob = scaler.fit_transform(scores)
        auroc = roc_auc_score(y_val, y_val_prob, multi_class='ovr')

    y_val_pred = pipe.predict(val_df[feature_candidates])
    print(f"\n=== {name} (Validation) ===")
    print(classification_report(y_val, y_val_pred))
    print("Macro AUROC:", auroc)

    results.append({'model': name, 'auroc_macro_val': auroc})

pd.DataFrame(results)


In [ ]:

# === 2) Lock best model and evaluate on test ===
# (Here we just pick XGB if present; in a real study, select by validation metric)
best_name = 'xgb' if 'xgb' in fitted else list(fitted.keys())[0]
best = fitted[best_name]

from sklearn.metrics import classification_report, ConfusionMatrixDisplay
y_test_pred = best.predict(test_df[[c for c in test_df.columns if c not in {target, 'patient_id'}]])
print(f"\n=== {best_name} (Test) ===")
print(classification_report(test_df[target], y_test_pred))

# Save model
best_path = PROC / f"model_{best_name}.joblib"
joblib.dump(best, best_path)
print("Saved model to", best_path)


## 3) Calibration & Decision Curves (binary example)

In [ ]:

# If you want to evaluate DCA and calibration on a *binary* task, pick one-vs-rest label:
binary_positive = 'severe'   # change to the class you care most about
y_train_bin = (train_df[target] == binary_positive).astype(int)
y_val_bin   = (val_df[target] == binary_positive).astype(int)
y_test_bin  = (test_df[target] == binary_positive).astype(int)

# Calibrated probability for the positive class
from sklearn.calibration import CalibratedClassifierCV
cal = CalibratedClassifierCV(fitted['logreg'] if 'logreg' in fitted else best, method='isotonic', cv=3)
cal.fit(train_df[[c for c in train_df.columns if c not in {target, 'patient_id'}]], y_train_bin)
probs = cal.predict_proba(test_df[[c for c in test_df.columns if c not in {target, 'patient_id'}]])[:,1]

# Reliability plot
from src.metrics import reliability_curve
from src.plotting import plot_reliability, plot_dca
rel = reliability_curve(y_test_bin, probs, n_bins=10)
plot_reliability(rel)

# Decision curve
from src.dca import decision_curve
dca = decision_curve(y_test_bin, probs)
plot_dca(dca)
